In [1]:
#imports from Sanzid
import h5py
import json
import matplotlib.pyplot as plt
import numpy as np
import numpy
import statistics
from numpy import loadtxt
import matplotlib.pyplot as plt
import pandas
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from statistics import stdev
import math
import h5py

import numpy as np
import time

from scipy.signal import butter,filtfilt
import sys
import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph
import seaborn as sns # used for plot interactive graph.
import pandas
import matplotlib.pyplot as plt

# from tsf.model import TransformerForecaster


# from tensorflow.keras.utils import np_utils
import itertools
###  Library for attention layers
import pandas as pd
import os
import numpy as np
#from tqdm import tqdm # Processing time measurement
from sklearn.model_selection import train_test_split

import statistics
import gc
import torch.nn.init as init

############################################################################################################################################################################
############################################################################################################################################################################

import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.utils.weight_norm as weight_norm
from sklearn.preprocessing import StandardScaler


import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
#from torchsummary import summary
from torch.nn.parameter import Parameter

#from torchsummary import summary



import torch.optim as optim
import gc

#from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler

#from google.colab import drive
#drive.mount('/content/drive')


In [3]:
#imports from dataloader final
import sys #you need this to add path to utils folder
sys.path.append('../utils') #now you can import from utils folder
# sys.path.append("../..")
from configs import config_general
from datasharders import datasharder_imu_joints
from datasets import ImuJointPairDataset
from data_utils import fft_filter_signal, wavelet_filter_signal
from model_utils import plot_predictions, plot_loss, train_model, evaluate_model
import torch
from torch.utils.data import DataLoader, random_split 
import matplotlib.pyplot as plt
from LSTM import DeepLSTMModel
import torch.nn as nn
import seaborn as sns

In [2]:
#imports for this notebook using sensor distillation utils
import sys #you need this to add path to utils folder
sys.path.append('../Sensor_distillation_utils') #now you can import from utils folder
from sensor_distillation import train_SD
from student_model import student
from teacher_model import teacher
from train import train_kinematics
from predictions import prediction_test

In [ ]:
#config from dataloader final
#some config errors

# Initialize config object
config = config_general(
    batch_size=64,
    epochs=1000,
    lr=0.002,
    scheduler=None,
    num_channels_imu=3,
    num_channels_joints=3,
    num_sessions=1,
    num_patients=3,
    seed=42,
    data_folder_name="../../datacollection/data",
    dataset_root="../../datasets",
    dataset_train_name="train_dataset",
    dataset_test_name="test_dataset",
    window_length=100,
    imu_transforms=[fft_filter_signal],
    joint_transforms=[], 
    hidden_size=256,
    num_layers=6,
    input_size=3,
    output_size=3
)


In [ ]:
#main function that runs everything from dataloader final
def main(config, remake_dataset):
    
    
    #device definition?
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model = DeepLSTMModel(input_size=config.input_size, hidden_size=config.hidden_size, num_layers=config.num_layers, output_size=config.output_size).to(device)
    criterion = nn.MSELoss() #changes depending on model
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr) #changes depending on model
   
    
    #remake_dataset false if no changes made, true if changes need to be made 
    #remake_dataset = True
    
    if(remake_dataset):
    
        #call datasharder on config 
        datasharder = datasharder_imu_joints(config, sample_rate=16000)
    
        #access data from datasharder
        training_data, testing_data = datasharder.load_data()

    
        #save windowed data to config specified destination
        datasharder.save_windowed_data(training_data, "train")
        datasharder.save_windowed_data(testing_data, "test")
    
   #split dataset into train and test
    dataset_train = ImuJointPairDataset(config, "train")
    dataset_test = ImuJointPairDataset(config, "test")
    #print(len(dataset_train)) 
    #print(len(dataset_test))
    #STOP 
    
    
    imu_data_tensor,joint_data_tensor=dataset_train.__getitem__(1)

    print(imu_data_tensor.shape, joint_data_tensor.shape)
    
    #convert to numpy
    imu_data = imu_data_tensor.numpy()
    joint_data = joint_data_tensor.numpy()
    sns.set(style="ticks")
    plt.plot(imu_data[:,1], label="imu")
    plt.plot(joint_data[:,1], label="joint")
    plt.legend()
    plt.show()
    
    train_size = int(0.75 * len(dataset_train))
    val_size = int(0.25 * len(dataset_train))
    
    train_dataset, val_dataset = random_split(dataset_train, [train_size, val_size])

    test_dataset = dataset_test
    
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)
    
    #train model
    train_losses, val_losses = train_model(model, train_loader, val_loader, config.epochs, criterion, optimizer, device)
    
    # Plot training and validation loss history
    plot_loss(train_losses, val_losses)
    
    # Evaluate model on validation and test sets and plot predictions
    val_inputs, val_targets, val_predictions = evaluate_model(model, val_loader, device)
    plot_predictions(val_inputs.squeeze(), val_targets.squeeze(), val_predictions.squeeze(), num_channels=3)

    test_inputs, test_targets, test_predictions = evaluate_model(model, test_loader, device)
    plot_predictions(test_inputs.squeeze(), test_targets.squeeze(), test_predictions.squeeze(), num_channels=3)

In [ ]:
#use main function accessing imu and joint data files directly from dataloader final
main(config, False)

In [3]:
#loading in data from lightweight kinematics net
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import os


#load all csvs from the folder and concatenate them into one dataframe by matching the columns
def load_data(path):
    df_list = []
    for file in os.listdir(path):
        if file.endswith('.csv'):
            data = pd.read_csv(os.path.join(path, file))
            df_list.append(data)
    df = pd.concat(df_list, ignore_index=True, sort=False)
    return df


# Index 0: pelvis_tilt
# Index 1: pelvis_list
# Index 2: pelvis_rotation
# Index 3: pelvis_tx
# Index 4: pelvis_ty
# Index 5: pelvis_tz
# Index 6: hip_flexion_r
# Index 7: hip_adduction_r
# Index 8: hip_rotation_r
# Index 9: knee_angle_r
# Index 10: knee_angle_r_beta
# Index 11: ankle_angle_r
# Index 12: subtalar_angle_r
# Index 13: mtp_angle_r
# Index 14: hip_flexion_l
# Index 15: hip_adduction_l
# Index 16: hip_rotation_l
# Index 17: knee_angle_l
# Index 18: knee_angle_l_beta
# Index 19: ankle_angle_l
# Index 20: subtalar_angle_l
# Index 21: mtp_angle_l
# Index 22: lumbar_extension
# Index 23: lumbar_bending
# Index 24: lumbar_rotation
# Index 25: arm_flex_r
# Index 26: arm_add_r
# Index 27: arm_rot_r
# Index 28: elbow_flex_r
# Index 29: pro_sup_r
# Index 30: wrist_flex_r
# Index 31: wrist_dev_r
# Index 32: arm_flex_l
# Index 33: arm_add_l
# Index 34: arm_rot_l
# Index 35: elbow_flex_l
# Index 36: pro_sup_l
# Index 37: wrist_flex_l
# Index 38: wrist_dev_l
# Index 39: time
# Index 40: 1
# Index 41: IM EMG1
# Index 42: IM EMG2
# Index 43: IM EMG3
# Index 44: IM EMG4
# Index 45: IM EMG5
# Index 46: IM EMG6
# Index 47: IM EMG7
# Index 48: IM EMG8
# Index 49: ACCX1
# Index 50: ACCY1
# Index 51: ACCZ1
# Index 52: GYROX1
# Index 53: GYROY1
# Index 54: GYROZ1
# Index 55: ACCX2
# Index 56: ACCY2
# Index 57: ACCZ2
# Index 58: GYROX2
# Index 59: GYROY2
# Index 60: GYROZ2
# Index 61: ACCX3
# Index 62: ACCY3
# Index 63: ACCZ3
# Index 64: GYROX3
# Index 65: GYROY3
# Index 66: GYROZ3
# Index 67: ACCX4
# Index 68: ACCY4
# Index 69: ACCZ4
# Index 70: GYROX4
# Index 71: GYROY4
# Index 72: GYROZ4
# Index 73: ACCX5
# Index 74: ACCY5
# Index 75: ACCZ5
# Index 76: GYROX5
# Index 77: GYROY5
# Index 78: GYROZ5
# Index 79: ACCX6
# Index 80: ACCY6
# Index 81: ACCZ6
# Index 82: GYROX6
# Index 83: GYROY6
# Index 84: GYROZ6
# Index 85: ACCX7
# Index 86: ACCY7
# Index 87: ACCZ7
# Index 88: GYROX7
# Index 89: GYROY7
# Index 90: GYROZ7
# Index 91: ACCX8
# Index 92: ACCY8
# Index 93: ACCZ8
# Index 94: GYROX8
# Index 95: GYROY8
# Index 96: GYROZ8


# gait_Net = train_kinematics_light(train_loader, lr,30,model,path+encoder+'_gait_net_kinematics_lightweight.pth')

In [4]:
# data loader will return [batch_size, sequence_length, input_dim] for 3 vector, acc and gyro as well as the target 
# from kinematics net notebook
class Sensor_Distillation_Dataset(Dataset):
    def __init__(self, data_acc, data_gyr, target, w, overlap=200):
        self.data_acc = data_acc
        self.data_gyr = data_gyr
        self.target = target
        self.w = w
        self.overlap = overlap

    def __len__(self):
        return (len(self.data_acc) - self.w) // (self.w - self.overlap) + 1

    def __getitem__(self, idx):
        start = idx * (self.w - self.overlap)
        end = start + self.w
        return (torch.tensor(self.data_acc[start:end]),
                torch.tensor(self.data_gyr[start:end]),
                torch.tensor(self.target[start:end]))  # Updated to return the entire target sequence

In [7]:

# Load and clean the combined CSV data
import sys #you need this to add path to utils folder
sys.path.append('../../datacollection') #now you can import from utils folder
from batch_process import combine_and_save_data

combine_and_save_data("../../datacollection/vicon/subject_1/motion", "../../datacollection/vicon/subject_1/processed", "../../datacollection/vicon/subject_1/combined")

combined_csv = "../../datacollection/vicon/subject_1/combined"

print(type(combined_csv))
# Check for NaN values
print(combined_csv.isnull().sum().sum())

# Drop NaN values
combined_csv = combined_csv.dropna()

print(combined_csv.isnull().sum().sum())

# Plot ACCX8, ACCY8, ACCZ8, GYROX8, GYROY8, GYROZ8
plt.plot(combined_csv['ACCX8'][:10000])
plt.plot(combined_csv['ACCY8'][:10000])
plt.plot(combined_csv['ACCZ8'][:10000])
plt.plot(combined_csv['GYROX8'][:10000])
plt.plot(combined_csv['GYROY8'][:10000])
plt.plot(combined_csv['GYROZ8'][:10000])
plt.show()

# Get two 3-channel arrays for ACC and GYRO for sensor 2
data_acc = combined_csv[['ACCX2', 'ACCY2', 'ACCZ2']].values
data_gyr = combined_csv[['GYROX2', 'GYROY2', 'GYROZ2']].values

# Normalize the data
scaler_acc = StandardScaler()
scaler_gyr = StandardScaler()
scaler_joints = StandardScaler()

data_acc = scaler_acc.fit_transform(data_acc)
data_gyr = scaler_gyr.fit_transform(data_gyr)
data_joints = combined_csv[['elbow_flex_r']].values
data_joints = scaler_joints.fit_transform(data_joints)

<class 'NoneType'>


AttributeError: 'NoneType' object has no attribute 'isnull'

In [ ]:
# Instantiate model from kinematics net notebook
model = Kinematics_lightweight(3, 3)

model.to('cuda')
# Calculate the total size of parameters in bytes
total_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
print(f'Total size in bytes: {total_size_bytes}')

# Convert total size in bytes to kilobytes
total_size_kb = total_size_bytes / 1024
print(f'Total size in kilobytes: {total_size_kb:.2f} KB')

# Example alternative model
# model = residual_net(6)

# Calculate the total size of parameters in bytes
total_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
print(f'Total size in bytes: {total_size_bytes}')

# Convert total size in bytes to kilobytes
total_size_kb = total_size_bytes / 1024
print(f'Total size in kilobytes: {total_size_kb:.2f} KB')

# Load and clean the combined CSV data
combined_csv = load_data("../../datacollection/vicon/subject_1/combined")

# Check for NaN values
print(combined_csv.isnull().sum().sum())

# Drop NaN values
combined_csv = combined_csv.dropna()

print(combined_csv.isnull().sum().sum())

# Plot ACCX8, ACCY8, ACCZ8, GYROX8, GYROY8, GYROZ8
plt.plot(combined_csv['ACCX8'][:10000])
plt.plot(combined_csv['ACCY8'][:10000])
plt.plot(combined_csv['ACCZ8'][:10000])
plt.plot(combined_csv['GYROX8'][:10000])
plt.plot(combined_csv['GYROY8'][:10000])
plt.plot(combined_csv['GYROZ8'][:10000])
plt.show()

# Get two 3-channel arrays for ACC and GYRO for sensor 2
data_acc = combined_csv[['ACCX2', 'ACCY2', 'ACCZ2']].values
data_gyr = combined_csv[['GYROX2', 'GYROY2', 'GYROZ2']].values

# Normalize the data
scaler_acc = StandardScaler()
scaler_gyr = StandardScaler()
scaler_joints = StandardScaler()

data_acc = scaler_acc.fit_transform(data_acc)
data_gyr = scaler_gyr.fit_transform(data_gyr)
data_joints = combined_csv[['elbow_flex_r']].values
data_joints = scaler_joints.fit_transform(data_joints)

# Create dataset instances with overlapping windows
dataset = KinematicsDataset(data_acc, data_gyr, data_joints, w=1000, overlap=500)

# Split the dataset into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [ ]:
#get first batch
data_acc, data_gyr, data_targets = next(iter(train_loader))

#print the shapes of the tensors
print(data_acc.shape)
print(data_gyr.shape)
print(data_targets.shape)


#plot the first 1000 samples of the first batch
plt.figure(figsize=(10, 5))
plt.plot(data_acc[0, :, 0], label='ACCX')
plt.plot(data_acc[0, :, 1], label='ACCY')
plt.plot(data_acc[0, :, 2], label='ACCZ')
plt.plot(data_gyr[0, :, 0], label='GYROX')
plt.plot(data_gyr[0, :, 1], label='GYROY')
plt.plot(data_gyr[0, :, 2], label='GYROZ')
plt.legend()
plt.title('Sensor Data')
plt.show()

#plot the first 1000 samples of the first batch
plt.figure(figsize=(10, 5))
plt.plot(data_targets[0,:1000])
plt.title('Target')
plt.show()

plt.plot(combined_csv[['elbow_flex_r']].values[:1000])
plt.show()

In [ ]:

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Train the model
gait_Net = train_kinematics_light(train_loader, val_loader, learn_rate=0.001, EPOCHS=100, model=model, filename='gait_net_kinematics_lightweight.pth', device=device, w=1000)


In [ ]:
#load in test data
combined_csv_test = load_data("../../datacollection/vicon/subject_2/combined")

# Check for NaN values
print(combined_csv_test.isnull().sum().sum())


# Drop NaN values
combined_csv_test = combined_csv_test.dropna()

print(combined_csv_test.isnull().sum().sum())

combined_csv_test



In [ ]:
# Plot ACCX8, ACCY8, ACCZ8, GYROX8, GYROY8, GYROZ8
plt.plot(combined_csv_test['ACCX10'][:10000])
plt.plot(combined_csv_test['ACCY10'][:10000])
plt.plot(combined_csv_test['ACCZ10'][:10000])
plt.plot(combined_csv_test['GYROX10'][:10000])
plt.plot(combined_csv_test['GYROY10'][:10000])
plt.plot(combined_csv_test['GYROZ10'][:10000])
plt.show()

# Get two 3-channel arrays for ACC and GYRO for sensor 2
data_acc_test = combined_csv_test[['ACCX2', 'ACCY2', 'ACCZ2']].values
data_gyr_test = combined_csv_test[['GYROX2', 'GYROY2', 'GYROZ2']].values

# Normalize the data
scaler_acc = StandardScaler()
scaler_gyr = StandardScaler()
scaler_joints = StandardScaler()

data_acc_test = scaler_acc.fit_transform(data_acc_test)
data_gyr_test = scaler_gyr.fit_transform(data_gyr_test)
data_targets_test = combined_csv_test[['elbow_flex_r']].values
data_targets_test = scaler_joints.fit_transform(data_targets_test)


# Create dataset instances with overlapping windows for test data
test_dataset = KinematicsDataset(data_acc_test, data_gyr_test, data_targets_test, w=1000, overlap=500)

# Create DataLoader for test data
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Load the trained model
model = Kinematics_lightweight(3, 3)
model.load_state_dict(torch.load('gait_net_kinematics_lightweight.pth'))
model.eval()
model.to(device)

# Evaluate the model on the test set
test_loss = 0
all_predictions = []
all_targets = []

criterion = RMSELoss()

with torch.no_grad():
    for data_acc, data_gyr, data_targets in test_loader:
        data_acc = data_acc.to(device).float()
        data_gyr = data_gyr.to(device).float()
        data_targets = data_targets.to(device).float().squeeze(-1)

        output = model(data_acc, data_gyr)
        loss = criterion(output, data_targets)
        test_loss += loss.item()

        all_predictions.append(output.cpu().numpy())
        all_targets.append(data_targets.cpu().numpy())

test_loss /= len(test_loader)
print(f'Test Loss: {test_loss:.4f}')

# Combine all predictions and targets
all_predictions = np.concatenate(all_predictions)
all_targets = np.concatenate(all_targets)

print(all_predictions.shape, all_targets.shape)  # Print shapes for debugging

# Plot individual graphs for each prediction and target
num_graphs = 10
for i in range(num_graphs):
    plt.figure(figsize=(10, 5))
    plt.plot(all_targets[i], label='True Values')
    plt.plot(all_predictions[i], label='Predictions')
    plt.xlabel('Time Frames')
    plt.ylabel('Elbow Flexion (r)')
    plt.legend()
    plt.title(f'Predictions vs True Values for Sample {i+1}')

    #set limits for y axis
    plt.ylim(-3,3)
    
   

    plt.show()



In [ ]:
print(all_predictions.shape, all_targets.shape)


In [ ]:

# Load the trained model
model = Kinematics_lightweight(3, 3)
model.load_state_dict(torch.load('gait_net_kinematics_lightweight.pth'))
model.eval()
model.to(device)

# Evaluate the model on the test set
test_loss = 0
all_predictions = []
all_targets = []

criterion = RMSELoss()

with torch.no_grad():
    for data_acc, data_gyr, data_targets in val_loader:
        data_acc = data_acc.to(device).float()
        data_gyr = data_gyr.to(device).float()
        data_targets = data_targets.to(device).float().squeeze(-1)

        output = model(data_acc, data_gyr)
        loss = criterion(output, data_targets)
        test_loss += loss.item()

        all_predictions.append(output.cpu().numpy())
        all_targets.append(data_targets.cpu().numpy())

test_loss /= len(test_loader)
print(f'Validation Loss: {test_loss:.4f}')

# Combine all predictions and targets
all_predictions = np.concatenate(all_predictions)
all_targets = np.concatenate(all_targets)

print(all_predictions.shape, all_targets.shape)  # Print shapes for debugging

# Calculate slopes
def calculate_slope(values):
    return np.gradient(values, axis=1)

slopes_predictions = calculate_slope(all_predictions)
slopes_targets = calculate_slope(all_targets)

# Plot individual graphs for each prediction and target
num_graphs = 10
for i in range(num_graphs):
    plt.figure(figsize=(10, 5))
    plt.plot(all_targets[i], label='True Values')
    plt.plot(all_predictions[i], label='Predictions')
    plt.xlabel('Time Frames')
    plt.ylabel('Elbow Flexion (r)')
    plt.legend()
    plt.title(f'Predictions vs Validation Values for Sample {i+1}')
    plt.ylim(-3, 3)
    plt.show()

In [ ]:
num_graphs = 10
for i in range(num_graphs):
    plt.figure(figsize=(10, 5))
    plt.plot(slopes_targets[i], label='True Values Slope')
    plt.plot(slopes_predictions[i], label='Predictions Slope')
    plt.xlabel('Time Frames')
    plt.ylabel('Slope')
    plt.legend()
    plt.title(f'Slope Comparison for Sample {i+1}')
    plt.ylim(-.005, .005)
    plt.show()

In [ ]:

# Function to calculate linear regression
def linear_regression(values):
    x = np.arange(values.shape[0])
    slope, intercept, _, _, _ = linregress(x, values)
    return slope * x + intercept, slope, intercept

# Calculate linear regression lines and get their slopes and intercepts
regression_lines_predictions = []
regression_lines_targets = []
slopes_predictions = []
intercepts_predictions = []
slopes_targets = []
intercepts_targets = []

for pred, target in zip(all_predictions, all_targets):
    reg_pred, slope_pred, intercept_pred = linear_regression(pred)
    reg_target, slope_target, intercept_target = linear_regression(target)
    regression_lines_predictions.append(reg_pred)
    regression_lines_targets.append(reg_target)
    slopes_predictions.append(slope_pred)
    intercepts_predictions.append(intercept_pred)
    slopes_targets.append(slope_target)
    intercepts_targets.append(intercept_target)

# Convert to numpy arrays
regression_lines_predictions = np.array(regression_lines_predictions)
regression_lines_targets = np.array(regression_lines_targets)

# Plot individual graphs for each prediction and target with linear regression lines
num_graphs = 10
for i in range(num_graphs):
    plt.figure(figsize=(10, 5))
    plt.plot(regression_lines_targets[i], label='True Values Regression', linestyle='--')
    plt.plot(regression_lines_predictions[i], label='Predictions Regression', linestyle='--')
    plt.xlabel('Time Frames')
    plt.ylabel('Elbow Flexion (r)')
    plt.legend()
    plt.title(f'Predictions vs Validation Values with Linear Regression for Sample {i+1}')
    plt.ylim(-3, 3)

    # Add text annotations for the linear regression formulas
    formula_target = f'True Values: y = {slopes_targets[i]:.2f}x + {intercepts_targets[i]:.2f}'
    formula_pred = f'Predictions: y = {slopes_predictions[i]:.2f}x + {intercepts_predictions[i]:.2f}'
    plt.text(0.05, 0.95, formula_target, transform=plt.gca().transAxes, fontsize=10, verticalalignment='top')
    plt.text(0.05, 0.90, formula_pred, transform=plt.gca().transAxes, fontsize=10, verticalalignment='top')

    plt.show()